/**********************************************************************************
 * Copyright (c) 2023 Process Systems Engineering (AVT.SVT), RWTH Aachen University
 *
 * This program and the accompanying materials are made available under the
 * terms of the Eclipse Public License 2.0 which is available at
 * http://www.eclipse.org/legal/epl-2.0.
 *
 * SPDX-License-Identifier: EPL-2.0
 *
 * The source code can be found here:
 * https://github.com/iAMB-RWTH-Aachen/Ustilago_maydis-GEM/tree/master/data/AcetateCofeed
 
 **********************************************************************************/




Date: 04.02.2023 <br/>
File name: Cofeed flux solutions.ipynb <br/>
Description: This script provides the FBA computations of the growth rates for different combinations of acetate and glucose uptake (cf. Figure 5 in the original manuscript).<br/>
@authors: Ulf Liebal, Manuel Boßmann, Anita Ziegler, Alexander Mitsos



In [1]:
import numpy as np
from os.path import join

# loading cobrapy, a library dedicated to the analysis of genome scale metabolic models
from cobra.io import read_sbml_model

In [2]:
#load iUma22v1.1.xml model
#https://github.com/iAMB-RWTH-Aachen/Ustilago_maydis-GEM
ModelFile = join('..', '..','model','iUma22v1.1.xml')
model=read_sbml_model(ModelFile)

In [33]:
# Definition of the number of carbon atoms participating in each reaction
# used to scale the reaction fluxes relative to C-mole    
RctFluxName = {'HEX1':6, 
         'MDH':4, 
         'THRALD':4,
         'TRANS-RXNTFS-1-ACET1223':2,
         'CSNATr':2, 
         'MDHm':4,
         'ACONTa':6, 
         'ISOCIT-CLEAV-RXN':6, 
         'MALSm':4, 
         'SUCDH_q6_m':4, 
         'Putative_Trans_Aconitate_Decarboxylase':1, 
         'EX_co2_e':1,
         'EX_itacon_e':5,
         # additional reactions that might be interesting to check for carbon fluxes
        #  'G6PDH':6, 
        #  'PPCKm':4,
        #  'PYK':3,
        #  'PCm':4,
        #  'TRANS-RXNTFS-7-ACET1018':2, 
        #  'PYRUVATE-CARBOXYLASE-RXN2':3,
        #  'TRANS_Malate_Phosphate_Shuttle':4, 
        #  'TRANS_Cis_Aconitate_Malate_Shuttle':4, 
        #  'HISTD':6, 
        #  'TRANS_Adenine_Nucleotide_Transporter':1,
        #  'BIOMASS_REACTION':1
         }

In [37]:
# Acetate uptake ratios of C-atoms
RatioNumb = [0,40,60]
# set objective function to itaconate secretion
model.objective = 'EX_itacon_e'
# maximum uptake rate of total C-mole
CFlux = 13.2
EXID = ['EX_ac_e','EX_glc__D_e']

# loop over acetate uptake ratios
for AceRat in RatioNumb:
    SubFlux = [AceRat*CFlux/(2*100),round((CFlux-AceRat*CFlux/100)/6,2)]
    ExchDict = {Substrate:Rate for Substrate, Rate in zip(EXID, SubFlux)}
    [setattr(model.reactions.get_by_id(key), 'lower_bound', -value) for key, value in ExchDict.items()]
    Flux = model.optimize()
    Ctot = np.abs(6*Flux['EX_glc__D_e'] + 2*Flux['EX_ac_e'])
    mySol = Flux.fluxes/Ctot*100
    # print reaction fluxes for each ratio of acetate uptake
    print('#################################################################################')
    print(f'Normalized 100 C-flux for ratio index {AceRat}: {Ctot:.2f}')
    [print(f'{model.reactions.get_by_id(Rct).id} \t {model.reactions.get_by_id(Rct).reaction} \t {round(mySol[Rct]*RctFluxName[Rct])}') for Rct in RctFluxName.keys()];


#################################################################################
Normalized 100 C-flux for ratio index 0: 13.20
HEX1 	 atp_c + glc__D_c --> adp_c + g6p_c + h_c 	 100
MDH 	 mal__L_c + nad_c <=> h_c + nadh_c + oaa_c 	 -76
THRALD 	 thr__L_c --> acald_c + gly_c 	 37
TRANS-RXNTFS-1-ACET1223 	 ac_e --> ac_c 	 0
PYRUVATE-CARBOXYLASE-RXN2 	 atp_m + hco3_m + pyr_m --> adp_m + h_m + oaa_m + pi_m 	 0
CSNATr 	 acrn_m + coa_m --> accoa_m + crn_m 	 19
MDHm 	 mal__L_m + nad_m <=> h_m + nadh_m + oaa_m 	 76
ACONTa 	 cit_m <=> acon_C_m + h2o_m 	 114
ISOCIT-CLEAV-RXN 	 icit_m --> glx_m + succ_m 	 0
MALSm 	 accoa_m + glx_m + h2o_m --> coa_m + h_m + mal__L_m 	 0
SUCDH_q6_m 	 q6_m + succ_m --> fum_m + q6h2_m 	 0
Putative_Trans_Aconitate_Decarboxylase 	 acon_T_c + h_c --> co2_c + itacon_c 	 19
EX_co2_e 	 co2_e <=>  	 5
EX_itacon_e 	 itacon_e -->  	 95
#################################################################################
Normalized 100 C-flux for ratio index 40: 13.20
HEX1 	 atp_c